In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

#df = pd.Dataframe()
df_train = pd.read_csv('./measuring-customer-happiness/train_hp.csv', encoding='utf-8')
print(df_train.head(3))

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

   User_ID                                        Description  \
0  id10326  The room was kind of clean but had a VERY stro...   
1  id10327  I stayed at the Crown Plaza April -- - April -...   
2  id10328  I booked this hotel through Hotwire at the low...   

        Browser_Used Device_Used Is_Response  
0               Edge      Mobile   not happy  
1  Internet Explorer      Mobile   not happy  
2            Mozilla      Tablet   not happy  


In [2]:
# Data Preprocessing

def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

X = []
sentences = list(df_train['Description'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [3]:
X[3]


'Stayed here with husband and sons on the way to an Alaska Cruise We all loved the hotel great experience Ask for room on the North tower facing north west for the best views We had high floor with stunning view of the needle the city and even the cruise ships We ordered room service for dinner so we could enjoy the perfect views Room service dinners were delicious too You are in perfect spot to walk everywhere so enjoy the city Almost forgot Heavenly beds were heavenly too '

In [4]:
# binary classification for happy and not_happy 

y = df_train['Is_Response']

y = np.array(list(map(lambda x: 1 if x=="happy" else 0, y)))

# 1) Simple Neural Network

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [6]:
# Prepare embedding layer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [7]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [8]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('./glove.twitter.27B/glove.twitter.27B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [9]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector


In [10]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          4154400   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 10001     
Total params: 4,164,401
Trainable params: 10,001
Non-trainable params: 4,154,400
_________________________________________________________________
None


# Model training

In [12]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 24916 samples, validate on 6229 samples
Epoch 1/6
24916/24916 [==============================] - 1s 35us/step - loss: 0.4973 - acc: 0.7614 - val_loss: 0.4491 - val_acc: 0.7939
Epoch 2/6
24916/24916 [==============================] - 1s 23us/step - loss: 0.3890 - acc: 0.8257 - val_loss: 0.4401 - val_acc: 0.7960
Epoch 3/6
24916/24916 [==============================] - 0s 18us/step - loss: 0.3524 - acc: 0.8468 - val_loss: 0.4452 - val_acc: 0.7987
Epoch 4/6
24916/24916 [==============================] - 1s 21us/step - loss: 0.3299 - acc: 0.8587 - val_loss: 0.4564 - val_acc: 0.7935
Epoch 5/6
24916/24916 [==============================] - 1s 22us/step - loss: 0.3107 - acc: 0.8696 - val_loss: 0.4907 - val_acc: 0.7821
Epoch 6/6
24916/24916 [==============================] - 0s 19us/step - loss: 0.2991 - acc: 0.8744 - val_loss: 0.4827 - val_acc: 0.7860


# 2) Convolutional Neural Network

In [13]:
from keras.layers.convolutional import Conv1D
model = Sequential()

embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [14]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          4154400   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 128)           64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 4,218,657
Trainable params: 64,257
Non-trainable params: 4,154,400
_________________________________________________________________
None


# Model Training & Evaluation 

In [15]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 24916 samples, validate on 6229 samples
Epoch 1/6
24916/24916 [==============================] - 9s 372us/step - loss: 0.4583 - acc: 0.7883 - val_loss: 0.3734 - val_acc: 0.8351
Epoch 2/6
24916/24916 [==============================] - 8s 328us/step - loss: 0.3329 - acc: 0.8577 - val_loss: 0.3472 - val_acc: 0.8462
Epoch 3/6
24916/24916 [==============================] - 8s 335us/step - loss: 0.2873 - acc: 0.8813 - val_loss: 0.3401 - val_acc: 0.8501
Epoch 4/6
24916/24916 [==============================] - 8s 335us/step - loss: 0.2478 - acc: 0.9040 - val_loss: 0.3333 - val_acc: 0.8512
Epoch 5/6
24916/24916 [==============================] - 8s 336us/step - loss: 0.2148 - acc: 0.9225 - val_loss: 0.3358 - val_acc: 0.8528
Epoch 6/6
7787/7787 [==============================] - 1s 132us/step


In [16]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])
print(score)

Test Score: 0.3248878301502698
Test Accuracy: 0.8606652112877903
[0.3248878301502698, 0.8606652112877903]


In [17]:
import pandas as pd

def test_data():
    
    df_test = pd.read_csv('./measuring-customer-happiness/test_hp.csv', encoding='utf-8')
    df_test = df_test['Description']
    pd.set_option('display.max_colwidth', 200)
    df_test.head()
    j=0
    for test_post in df_test:
        if(j<=100):
            test_post = tokenizer.texts_to_sequences(test_post) 
            flat_list = []
            for sublist in instance:
                for item in sublist:
                    flat_list.append(item)
            flat_list = [flat_list]
            test_post = pad_sequences(flat_list, padding='post', maxlen=maxlen)
            model.predict(test_post)  
            ++j
        
test_data()

NameError: name 'instance' is not defined

## Load Facebook Comments

In [ ]:
import json
import requests
import pandas as pd
import datetime

class FacebookApi:

    ## below facebook api call


    def callFacebookApi(self):
        
        url_long = "https://graph.facebook.com/v5.0/me?fields=id%2Cname%2Cposts%7Bcomments%7D&access_token=EAAlcIv35CUUBANREEygggKozZBFTNubNFhwuDn0u3MDI1jHz4PYRGirDFxz7MoSMTz3AHu6ZCQdT0oBp0cM3a30fGzw1pfb27C6H5OGn5jxiV49TqK8yaBiZA6DKZAeR3r0yzGGFFMGZAGmf6lMDeh8elMMGFZA5z4pk8KfQ2uiDZAZB67gt1nfh"

        response = requests.get(url_long)
        json_data = json.loads(response.text)
        #df = pd.DataFrame(columns=['comment', 'person', 'time'])
        dictFb = {'comment': [], 'person':[], 'time':[]}
        for i in range(len(json_data['posts']['data'])-1):
            dictFb['comment'].append(json_data['posts']['data'][i]['comments']['data'][i]['message'])
            dictFb['person'].append(json_data['posts']['data'][i]['comments']['data'][i]['from']['name'])
            time = json_data['posts']['data'][i]['comments']['data'][i]['created_time']
            
            datetime.datetime.strptime(df['time'], "%Y-%m-%dT%H:%M:%S+0000")
            dictFb['time'].append(json_data['posts']['data'][i]['comments']['data'][i]['created_time'])
            #df.append({'comment': comment, 'person':person, 'time':time}, ignore_index=True)
        df = pd.DataFrame(dictFb,columns=['comment', 'person', 'time'])

        return df
    

In [ ]:
fb =FacebookApi()
fb.callFacebookApi()
print(fb.callFacebookApi())

In [ ]:
def findDay(date): 
    born = datetime.datetime.strptime(date, '%d %m %Y').weekday() 
    return (calendar.day_name[born]) 
  
# Driver program 
date = '03 02 2019'
print(findDay(date)) 

# Load facebook comments

# Predict sentiments

In [ ]:
instance = X[57]

print(instance)


####  - convert review into numeric form (using the tokenizer)
####  - text_to_sequences method will convert the sentence into its numeric counter part
####  - positive = 1, negative = 0
####  - sigmoid function predicts floating value between 0 and 1. 
####  - value < 0.5 = negative sentiment 
####  - value > 0.5 = positive sentiment 


In [ ]:
instance = tokenizer.texts_to_sequences(instance)

flat_list = []
for sublist in instance:
    for item in sublist:
        flat_list.append(item)

flat_list = [flat_list]

instance = pad_sequences(flat_list, padding='post', maxlen=maxlen)

model.predict(instance)

# Send email

In [ ]:
import smtplib, ssl

port = 587  # For starttls
smtp_server = "smtp.gmail.com"
receiver_email = "burghard.lachmann+kunde@gmail.com"
sender_email = "burghard.lachmann@gmail.com"
password = 'Brunhilde'
message = """
Report von HH Analytica
"""

context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
   # server.ehlo()  # Can be omitted
    server.starttls(context=context)
    #server.ehlo()  # Can be omitted
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

In [ ]:
from smtplib import SMTP
with SMTP("https://www.web.de") as smtp:
    smtp.noop()